In [22]:
import pandas as pd
import os

In [23]:
header_names = ["ticker","price","time","meta_signal","meta_previous",
                "h","d","w","m",
                "hp","dp","wp","mp",
                "h_rsi","h_stochrsi","h_stoch","h_ult","h_macd","h_hull",
                "d_rsi","d_stochrsi","d_stoch","d_ult","d_macd","d_hull",
                "w_rsi","w_stochrsi","w_stoch","w_ult","w_macd","w_hull",
                "m_rsi","m_stochrsi","m_stoch","m_ult","m_macd","m_hull",
                "h_divergence","d_divergence","w_divergence","m_divergence"
                ]

In [24]:
# first, combine dates into single files
files = os.listdir('./')
csv_files = [f for f in files if "signals.20" in f]
csv_files.sort()
csv_files

frames={}
lists={}

# first build the list of dataframes
for file_ in csv_files:
    # split date out of file names
    file_split=file_.split('.', 1)
    year_split=file_split[1].split('-', 1)
    month_split=year_split[1].split('-', 1)
    day_split=month_split[1].split('-', 1)
    year=year_split[0]
    month=month_split[0]
    day=day_split[0]
    
    date=year+'-'+month+'-'+day
    if (date not in lists):
        lists[date] = []
        
    lists[date].append(pd.read_csv(file_,index_col=None, header=None))
    
for date in lists:
    print(date)
    frame = pd.concat(lists[date])
    frame.columns = header_names
    frame.sort_values(['ticker', 'time'], inplace=True)
    frame.to_csv('./signals-'+date+'.csv', header=None, index=None)

2018-09-08
2018-09-09


In [25]:
files = os.listdir('./')
csv_files = [f for f in files if "signals-20" in f]
csv_files

['signals-2018-08-28.csv',
 'signals-2018-08-29.csv',
 'signals-2018-09-08.csv',
 'signals-2018-09-09.csv',
 'signals-2018-09-01.csv',
 'signals-2018-09-02.csv',
 'signals-2018-09-03.csv',
 'signals-2018-09-07.csv',
 'signals-2018-09-06.csv',
 'signals-2018-09-04.csv',
 'signals-2018-09-05.csv',
 'signals-2018-08-22.csv',
 'signals-2018-08-23.csv',
 'signals-2018-08-24.csv',
 'signals-2018-08-30.csv',
 'signals-2018-08-31.csv',
 'signals-2018-08-27.csv']

In [26]:
frame = pd.DataFrame()
list_ = []
for file_ in csv_files:
    df = pd.read_csv(file_,index_col=None, header=None)
    list_.append(df)
frame = pd.concat(list_)
frame.columns = header_names
frame.sort_values(['ticker', 'time'], inplace=True)
frame.head(5)

,ticker,price,time,meta_signal,meta_previous,h,d,w,m,hp,...,m_rsi,m_stochrsi,m_stoch,m_ult,m_macd,m_hull,h_divergence,d_divergence,w_divergence,m_divergence
0,AMEX-BTG,2.310,1534976694686,-1,0,-1,-1,-1,-1,0.0,...,45.06,-1.0,-1,1,1,-1.0,0,0,0,0
0,AMEX-BTG,2.240,1535032150741,-1,0,-1,-1,-1,-1,0.0,...,43.23,-1.0,-1,-1,-1,-1.0,0,0,0,0
1,AMEX-BTG,2.235,1535036020573,-1,0,-1,-1,-1,-1,0.0,...,42.94,-1.0,-1,-1,-1,-1.0,0,0,0,0
2,AMEX-BTG,2.210,1535039443967,-1,0,-1,-1,-1,-1,0.0,...,42.24,-1.0,-1,-1,-1,-1.0,0,0,0,0
3,AMEX-BTG,2.215,1535042714300,-1,0,-1,-1,-1,-1,0.0,...,42.24,-1.0,-1,-1,-1,-1.0,0,0,0,0


In [27]:
print('shape', frame.shape)
print("unique tickers: {} ".format(frame.ticker.nunique()))
tickers = set(frame.ticker.unique())

shape (14371, 41)
unique tickers: 180 


In [28]:
# split ticker into market-name
# frame['market'], frame['name'] = frame.ticker.str.split('-', 1).str
# frame.sort_values(['name', 'time'], inplace=True)
# frame.head(2)

In [29]:
# failures to get price (numeric conversion)
frame[frame.price.isnull()]

,ticker,price,time,meta_signal,meta_previous,h,d,w,m,hp,...,m_rsi,m_stochrsi,m_stoch,m_ult,m_macd,m_hull,h_divergence,d_divergence,w_divergence,m_divergence


In [30]:
# drop any NaN/null values
# this will happen the first time we run queries against a ticker, until we get diffs on the prev meta signals
# frame = frame.dropna()

# replace NaN values (nicer)
frame = frame.replace('NaN', '/')
print('shape', frame.shape)
print("unique tickers: {} ".format(frame.ticker.nunique()))
print("missing", set(tickers) - set(frame.ticker.unique()))
abb = frame[frame.ticker.astype(str).str.match("NYSE-ABB")]
abb.head(1)

shape (14371, 41)
unique tickers: 180 
missing set()


,ticker,price,time,meta_signal,meta_previous,h,d,w,m,hp,...,m_rsi,m_stochrsi,m_stoch,m_ult,m_macd,m_hull,h_divergence,d_divergence,w_divergence,m_divergence
718,NYSE-ABB,23.72,1535494347788,1,0,1,1,1,1,/,...,44.71,-1.0,-1,-1,1,1.0,0,0,0,0


In [31]:
test = frame[frame.ticker.astype(str).str.match("TSX-ENB")]
test.head(1)
test.head(1).values

array([['TSX-ENB', 36.18, 1534978556364, 1, 1, 1, 1, 1, -1, 2.0, 2.0, 0.0,
        0.0, 54.37, 63.14, 72.66, 50.25, 1, 57.01, 61.58, 78.65, 51.75, -1,
        59.1, 94.41, 93.96, 58.4, 1, 47.25, 94.92, 55.01, 48.3, -1.0, -1,
        -1, -1, 1.0, 0, 0, 0, 0]], dtype=object)

In [32]:
# newer tickers (e.g. IGRO, BOTZ, etc) will not have enough data for monthly signals
# script adds "/" in these cases
# we need to convert those to 0 for the ML engine
frame = frame.replace('/', 0)
print('shape', frame.shape)
print("unique tickers: {} ".format(frame.ticker.nunique()))
print("missing", set(tickers) - set(frame.ticker.unique()))
botz = frame[frame.ticker.astype(str).str.match("NASDAQ-BOTZ")]
botz.head(1)

shape (14371, 41)
unique tickers: 180 
missing set()


,ticker,price,time,meta_signal,meta_previous,h,d,w,m,hp,...,m_rsi,m_stochrsi,m_stoch,m_ult,m_macd,m_hull,h_divergence,d_divergence,w_divergence,m_divergence
33,NASDAQ-BOTZ,22.29,1534976650054,1,0,1,1,-1,1,-1.0,...,0.0,0.0,-1,1,1,1.0,0,0,0,0


In [33]:
# more failures to get price
frame[frame.price == 0]

,ticker,price,time,meta_signal,meta_previous,h,d,w,m,hp,...,m_rsi,m_stochrsi,m_stoch,m_ult,m_macd,m_hull,h_divergence,d_divergence,w_divergence,m_divergence


In [34]:
# remove them if they exist
frame = frame[frame.price!=0]
print('shape', frame.shape)
print("unique tickers: {} ".format(frame.ticker.nunique()))
print("missing", set(tickers) - set(frame.ticker.unique()))

shape (14371, 41)
unique tickers: 180 
missing set()


In [35]:
abbv = frame[frame.ticker.astype(str).str.match("NYSE-ABBV")]
abbv

,ticker,price,time,meta_signal,meta_previous,h,d,w,m,hp,...,m_rsi,m_stochrsi,m_stoch,m_ult,m_macd,m_hull,h_divergence,d_divergence,w_divergence,m_divergence
80,NYSE-ABBV,97.6400,1534986940463,1,1,1,1,1,1,2.0,...,44.62,-1.0,1,-1,-1,1.0,0,0,0,0
685,NYSE-ABBV,97.3000,1535031845738,1,1,1,1,1,1,2.0,...,44.36,-1.0,-1,-1,-1,1.0,0,0,0,0
686,NYSE-ABBV,97.4000,1535035671968,1,1,1,1,1,1,2.0,...,44.45,-1.0,1,-1,-1,1.0,0,0,0,0
687,NYSE-ABBV,97.0500,1535039099282,1,1,0,1,1,1,1.0,...,44.17,-1.0,-1,-1,-1,1.0,0,0,0,0
688,NYSE-ABBV,96.9000,1535042394468,1,1,0,1,0,1,1.0,...,44.06,-1.0,-1,-1,-1,1.0,0,0,0,0
689,NYSE-ABBV,97.1600,1535045639362,1,1,1,1,1,1,0.0,...,44.25,-1.0,1,-1,-1,1.0,0,0,0,0
690,NYSE-ABBV,97.0800,1535048868582,1,1,-1,1,1,1,1.0,...,44.17,-1.0,-1,-1,-1,1.0,0,0,0,0
691,NYSE-ABBV,97.1100,1535052105565,1,0,1,1,1,1,-1.0,...,44.22,-1.0,-1,-1,-1,1.0,0,0,0,0
692,NYSE-ABBV,97.1500,1535055309868,1,0,1,1,1,1,-1.0,...,44.25,-1.0,1,-1,-1,1.0,0,0,0,0
693,NYSE-ABBV,97.1500,1535058535964,1,0,1,1,1,1,-1.0,...,44.25,-1.0,1,-1,-1,1.0,0,0,0,0


In [36]:
# remove duplicates
df = frame.drop_duplicates(["ticker","price","meta_signal","meta_previous",
                "h","d","w","m",
                "hp","dp","wp","mp",
                "h_rsi","h_stochrsi","h_stoch","h_ult","h_macd","h_hull",
                "d_rsi","d_stochrsi","d_stoch","d_ult","d_macd","d_hull",
                "w_rsi","w_stochrsi","w_stoch","w_ult","w_macd","w_hull",
                "m_rsi","m_stochrsi","m_stoch","m_ult","m_macd","m_hull",
                "h_divergence","d_divergence","w_divergence","m_divergence"
            ])
df[df.ticker.astype(str).str.match("NYSE-ABBV")]

,ticker,price,time,meta_signal,meta_previous,h,d,w,m,hp,...,m_rsi,m_stochrsi,m_stoch,m_ult,m_macd,m_hull,h_divergence,d_divergence,w_divergence,m_divergence
80,NYSE-ABBV,97.6400,1534986940463,1,1,1,1,1,1,2.0,...,44.62,-1.0,1,-1,-1,1.0,0,0,0,0
685,NYSE-ABBV,97.3000,1535031845738,1,1,1,1,1,1,2.0,...,44.36,-1.0,-1,-1,-1,1.0,0,0,0,0
686,NYSE-ABBV,97.4000,1535035671968,1,1,1,1,1,1,2.0,...,44.45,-1.0,1,-1,-1,1.0,0,0,0,0
687,NYSE-ABBV,97.0500,1535039099282,1,1,0,1,1,1,1.0,...,44.17,-1.0,-1,-1,-1,1.0,0,0,0,0
688,NYSE-ABBV,96.9000,1535042394468,1,1,0,1,0,1,1.0,...,44.06,-1.0,-1,-1,-1,1.0,0,0,0,0
689,NYSE-ABBV,97.1600,1535045639362,1,1,1,1,1,1,0.0,...,44.25,-1.0,1,-1,-1,1.0,0,0,0,0
690,NYSE-ABBV,97.0800,1535048868582,1,1,-1,1,1,1,1.0,...,44.17,-1.0,-1,-1,-1,1.0,0,0,0,0
691,NYSE-ABBV,97.1100,1535052105565,1,0,1,1,1,1,-1.0,...,44.22,-1.0,-1,-1,-1,1.0,0,0,0,0
692,NYSE-ABBV,97.1500,1535055309868,1,0,1,1,1,1,-1.0,...,44.25,-1.0,1,-1,-1,1.0,0,0,0,0
688,NYSE-ABBV,96.9400,1535120333448,0,1,-1,1,0,1,1.0,...,44.09,-1.0,-1,-1,-1,1.0,0,0,0,0


In [37]:
print('shape', frame.shape)
print("unique tickers: {} ".format(frame.ticker.nunique()))

shape (14371, 41)
unique tickers: 180 


In [38]:
# add meta movement indicator
df['meta_move'] = df['meta_signal'].values - df['meta_previous'].values
df.head(3)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ticker,price,time,meta_signal,meta_previous,h,d,w,m,hp,...,m_stochrsi,m_stoch,m_ult,m_macd,m_hull,h_divergence,d_divergence,w_divergence,m_divergence,meta_move
0,AMEX-BTG,2.310,1534976694686,-1,0,-1,-1,-1,-1,0.0,...,-1.0,-1,1,1,-1.0,0,0,0,0,-1
0,AMEX-BTG,2.240,1535032150741,-1,0,-1,-1,-1,-1,0.0,...,-1.0,-1,-1,-1,-1.0,0,0,0,0,-1
1,AMEX-BTG,2.235,1535036020573,-1,0,-1,-1,-1,-1,0.0,...,-1.0,-1,-1,-1,-1.0,0,0,0,0,-1


In [39]:
# round to 3 decimals (python tweaks the numbers)
df.price = df.price.round(3)
df[df.ticker=='NASDAQ-HMNY']

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,ticker,price,time,meta_signal,meta_previous,h,d,w,m,hp,...,m_stochrsi,m_stoch,m_ult,m_macd,m_hull,h_divergence,d_divergence,w_divergence,m_divergence,meta_move
49,NASDAQ-HMNY,0.030,1534983502339,-1,0,-1,-1,-1,-2,0.0,...,-1.0,1,1,1,1.0,0,0,0,0,-1
425,NASDAQ-HMNY,0.030,1535033202901,-1,0,-1,-1,-1,-2,0.0,...,-1.0,1,1,1,1.0,0,0,0,0,-1
426,NASDAQ-HMNY,0.028,1535037142410,-1,0,-1,-1,-1,-2,0.0,...,-1.0,-1,1,1,1.0,0,0,0,0,-1
427,NASDAQ-HMNY,0.023,1535040526677,-1,0,-1,-1,-1,-2,0.0,...,-1.0,-1,-1,1,1.0,0,0,0,0,-1
428,NASDAQ-HMNY,0.022,1535043779500,-1,0,-1,-1,-1,-2,0.0,...,-1.0,-1,-1,1,1.0,0,0,0,0,-1
429,NASDAQ-HMNY,0.022,1535047010532,-1,0,-1,-1,-1,-2,0.0,...,-1.0,-1,-1,1,1.0,0,0,0,0,-1
430,NASDAQ-HMNY,0.022,1535050262705,-1,0,-1,-1,-1,-2,0.0,...,-1.0,-1,-1,1,1.0,0,0,0,0,-1
431,NASDAQ-HMNY,0.020,1535053482866,-1,0,-1,-1,-1,-2,0.0,...,-1.0,-1,-1,1,1.0,0,0,0,0,-1
432,NASDAQ-HMNY,0.020,1535056689683,-1,0,-1,-1,-1,-2,0.0,...,-1.0,-1,-1,1,1.0,0,0,0,0,-1
422,NASDAQ-HMNY,0.019,1535118109283,-1,0,-1,-1,-1,-2,0.0,...,-1.0,-1,-1,1,1.0,0,0,0,0,-1


In [40]:
df['Meta OK'] = (df.meta_signal==1) | (df.meta_signal==2) | (df.meta_signal==0)
df['Meta Good'] = (df.meta_signal==1) | (df.meta_signal==2)
df['Meta ⬆️'] = df.meta_move > 0
df['Meta ⬆️⬆️'] = df.meta_move > 1
df['Meta Bad'] = (df.meta_signal==-1) | (df.meta_signal==-2)
df['Meta ⬇️'] = df.meta_move < 0
df['Meta ⬇️⬇️'] = df.meta_move < -1

df['RSI Over Short'] = ((df['h_rsi'] > 70) & (df['d_rsi'] > 70)).astype(int)
df['RSI Over Long'] = ((df['w_rsi'] > 70) & (df['m_rsi'] > 70)).astype(int)
df['RSI Over All'] = (df['RSI Over Short'] & df['RSI Over Long']).astype(int)
df['RSI Under Short'] = ((df['h_rsi'] < 30) & (df['d_rsi'] < 30)).astype(int)
df['RSI Under Long'] = ((df['w_rsi'] < 30) & (df['m_rsi'] < 30)).astype(int)
df['RSI Under All'] = (df['RSI Under Short'] & df['RSI Under Long']).astype(int)

df['StochRSI Over Short'] = ((df['h_stochrsi'] > 70) & (df['d_stochrsi'] > 70)).astype(int)
df['StochRSI Over Long'] = ((df['w_stochrsi'] > 70) & (df['m_stochrsi'] > 70)).astype(int)
df['StochRSI Over All'] = (df['StochRSI Over Short'] & df['StochRSI Over Long']).astype(int)
df['StochRSI Under Short'] = ((df['h_stochrsi'] < 30) & (df['d_stochrsi'] < 30)).astype(int)
df['StochRSI Under Long'] = ((df['w_stochrsi'] < 30) & (df['m_stochrsi'] < 30)).astype(int)
df['StochRSI Under All'] = (df['StochRSI Under Short'] & df['StochRSI Under Long']).astype(int)

df['Stoch Over Short'] = ((df['h_stoch'] > 80) & (df['d_stoch'] > 80)).astype(int)
df['Stoch Over Long'] = ((df['w_stoch'] > 80) & (df['m_stoch'] > 80)).astype(int)
df['Stoch Over All'] = (df['Stoch Over Short'] & df['Stoch Over Long']).astype(int)
df['Stoch Under Short'] = ((df['h_stoch'] < 20) & (df['d_stoch'] < 20)).astype(int)
df['Stoch Under Long'] = ((df['w_stoch'] < 20) & (df['m_stoch'] < 20)).astype(int)
df['Stoch Under All'] = (df['Stoch Under Short'] & df['Stoch Under Long']).astype(int)

df['Ult Over Short'] = ((df['h_ult'] > 70) & (df['d_ult'] > 70)).astype(int)
df['Ult Over Long'] = ((df['w_ult'] > 70) & (df['m_ult'] > 70)).astype(int)
df['Ult Over All'] = (df['Ult Over Short'] & df['Ult Over Long']).astype(int)
df['Ult Under Short'] = ((df['h_ult'] < 30) & (df['d_ult'] < 30)).astype(int)
df['Ult Under Long'] = ((df['w_ult'] < 30) & (df['m_ult'] < 30)).astype(int)
df['Ult Under All'] = (df['Ult Under Short'] & df['Ult Under Long']).astype(int)

df.head(1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ticker,price,time,meta_signal,meta_previous,h,d,w,m,hp,...,Stoch Over All,Stoch Under Short,Stoch Under Long,Stoch Under All,Ult Over Short,Ult Over Long,Ult Over All,Ult Under Short,Ult Under Long,Ult Under All
0,AMEX-BTG,2.31,1534976694686,-1,0,-1,-1,-1,-1,0.0,...,0,0,1,0,0,0,0,0,0,0


In [41]:
df.columns

Index(['ticker', 'price', 'time', 'meta_signal', 'meta_previous', 'h', 'd',
       'w', 'm', 'hp', 'dp', 'wp', 'mp', 'h_rsi', 'h_stochrsi', 'h_stoch',
       'h_ult', 'h_macd', 'h_hull', 'd_rsi', 'd_stochrsi', 'd_stoch', 'd_ult',
       'd_macd', 'd_hull', 'w_rsi', 'w_stochrsi', 'w_stoch', 'w_ult', 'w_macd',
       'w_hull', 'm_rsi', 'm_stochrsi', 'm_stoch', 'm_ult', 'm_macd', 'm_hull',
       'h_divergence', 'd_divergence', 'w_divergence', 'm_divergence',
       'meta_move', 'Meta OK', 'Meta Good', 'Meta ⬆️', 'Meta ⬆️⬆️', 'Meta Bad',
       'Meta ⬇️', 'Meta ⬇️⬇️', 'RSI Over Short', 'RSI Over Long',
       'RSI Over All', 'RSI Under Short', 'RSI Under Long', 'RSI Under All',
       'StochRSI Over Short', 'StochRSI Over Long', 'StochRSI Over All',
       'StochRSI Under Short', 'StochRSI Under Long', 'StochRSI Under All',
       'Stoch Over Short', 'Stoch Over Long', 'Stoch Over All',
       'Stoch Under Short', 'Stoch Under Long', 'Stoch Under All',
       'Ult Over Short', 'Ult Over L

In [42]:
# export new, cleaned db file
df.to_csv('./signals.csv', header=None, index=None)